In [1]:
import os
import sys
from pathlib import Path 

In [2]:
%pwd

'c:\\Users\\fifty\\OneDrive\\Desktop\\AI - Data Science\\Malicious_QR_Code_Detection - 0.2\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\fifty\\OneDrive\\Desktop\\AI - Data Science\\Malicious_QR_Code_Detection - 0.2'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    
    csv_dir: Path
    csv_file: str

In [6]:
from src.MaliciousQRCodeDetection.constants import *
from src.MaliciousQRCodeDetection.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH):
        
        self.config = read_yaml(config_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            csv_dir = config.csv_dir,
            csv_file = config.csv_file
        )

        return data_ingestion_config
    

In [8]:
from src.MaliciousQRCodeDetection.logging.logger import logger
from src.MaliciousQRCodeDetection.exception import MaliciousQRException
from src.MaliciousQRCodeDetection.utils.common import get_size
import urllib.request as request

In [8]:
import os
import pymongo
import certifi
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from MaliciousQRCodeDetection.exception import MaliciousQRException
from MaliciousQRCodeDetection.logging.logger import logger
from MaliciousQRCodeDetection.constants import db_name, mongodb_url_key, collection_name
import sys

In [9]:
class MongoDBClient:
    """
    MongoDBClient: Handles MongoDB connection and data export.

    Methods:
    - __init__: Initializes MongoDB client connection.
    - export_collection_as_dataframe: Exports a MongoDB collection to a Pandas DataFrame.
    """
    def __init__(self, database_name=db_name, collection_name=collection_name) -> None:
        try:
            # Load environment variables
            load_dotenv('project.env')
            mongo_db_url = os.getenv(mongodb_url_key)
            if not mongo_db_url:
                raise ValueError(f"Environment variable '{mongodb_url_key}' is not set.")
            
            # Create MongoDB client
            self.client = pymongo.MongoClient(mongo_db_url, tlsCAFile=certifi.where())
            self.database = self.client[database_name]
            self.collection_name = collection_name
            logger.info("MongoDB connection established successfully.")
        except Exception as e:
            raise MaliciousQRException(e,sys)

    def export_collection_as_dataframe(self) -> pd.DataFrame:
        """
        Exports the specified collection as a Pandas DataFrame.
        
        Returns:
            pd.DataFrame: The MongoDB collection as a DataFrame.
        """
        try:
            collection = self.database[self.collection_name]
            data = list(collection.find())
            
            # Convert to DataFrame and clean up
            df = pd.DataFrame(data)
            if "_id" in df.columns:
                df.drop(columns=["_id"], inplace=True)
            df.replace({"na": np.nan}, inplace=True)
            return df
        except Exception as e:
            raise MaliciousQRException(e,sys)

In [10]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        '''
        param data_ingestion_config: configuration for data ingestion
        '''

        try:
            self.config = config
        except Exception as e:
            raise MaliciousQRException(e,sys)

    def export_data_into_data_store(self):
        try:
            data = MongoDBClient()
            df = data.export_collection_as_dataframe()

            if not os.path.exists(self.config.csv_file):
                df.to_csv(self.config.csv_file,index=False)
                logger.info('Csv file successully saved in artifacts data ingestion')
            else:
                logger.info('CSV file already exists in artifacts data ingestion')

        except Exception as e:
            raise MaliciousQRException(e,sys)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.export_data_into_data_store()
except Exception as e:
    logger.error(f'Unexpected error occured while downloading file: {e}')
    raise MaliciousQRException(e,sys)

[ 2025-01-23 06:29:09,187] 17 root - INFO - yaml file: config\config.yaml loaded successfully
[ 2025-01-23 06:29:09,187] 31 root - INFO - Created directory at: artifacts
[ 2025-01-23 06:29:09,189] 31 root - INFO - Created directory at: artifacts/data_ingestion
[ 2025-01-23 06:29:13,613] 21 root - INFO - MongoDB connection established successfully.
[ 2025-01-23 06:29:31,213] 21 root - INFO - CSV file already exists in artifacts data ingestion
